# M3D-NCA: Robust 3D Segmentation with Built-in Quality Control
### John Kalkhof, Anirban Mukhopadhyay
__TBD__



***

## __The Backbone Model__
<div>
<img src="src/images/model_M3DNCA.png" width="600"/>
</div>

In [2]:
!pip install nibabel

     ---------------------------------------- 3.3/3.3 MB 5.0 MB/s eta 0:00:00


In [4]:
!pip install torchio

     -------------------------------------- 172.8/172.8 kB 1.3 MB/s eta 0:00:00
     -------------------------------------- 110.0/110.0 kB 3.2 MB/s eta 0:00:00
  Using cached Deprecated-1.2.13-py2.py3-none-any.whl (9.6 kB)
     -------------------------------------- 27.0/27.0 MB 588.4 kB/s eta 0:00:00
  Using cached rich-12.6.0-py3-none-any.whl (237 kB)
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)


In [29]:
!pip install aim==3.7.0

     -------------------------------------- 15.3/15.3 MB 491.9 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached aimrecords-0.0.7-py2.py3-none-any.whl (28 kB)


ERROR: Could not find a version that satisfies the requirement aimrocks==0.1.0 (from aim) (from versions: 0.2.0)
ERROR: No matching distribution found for aimrocks==0.1.0


## _1. Imports_

In [14]:
import torch
from src.datasets.Nii_Gz_Dataset_3D import Dataset_NiiGz_3D
from src.models.Model_BasicNCA3D import BasicNCA3D
from src.losses.LossFunctions import DiceFocalLoss
from src.utils.Experiment import Experiment
from src.agents.Agent_M3D_NCA import Agent_M3D_NCA

ModuleNotFoundError: No module named 'aim'

## _2. Configure experiment_
- __AutoReload__
    - If an experiment already exists in _model\_path_ the config will __always__ be overwritten with the existing one
    - Additionally if the model has been saved previously, this state will be reloaded
- Download _prostate_ data from 'http://medicaldecathlon.com/' and adapt 'img_path' and 'label_path'

In [12]:
config = [{
    'img_path': r"C:\Users\AVNI\Desktop\6th SEM\Neullar Cellular Automata\NCA-main\3d_images\imgs",
    'label_path': r"C:\Users\AVNI\Desktop\6th SEM\Neullar Cellular Automata\NCA-main\3d_images\msks",
    'name': r'M3D_NCA_Run10',
    'device':"cpu",
    'unlock_CPU': True,
    # Optimizer
    'lr': 16e-4,
    'lr_gamma': 0.9999,
    'betas': (0.9, 0.99),
    # Training
    'save_interval': 10,
    'evaluate_interval': 10,
    'n_epoch': 3000,
    'batch_duplication': 1,
    # Model
    'channel_n': 16,        # Number of CA state channels
    'inference_steps': [10, 10],
    'cell_fire_rate': 0.5,
    'batch_size': 4,
    'input_channels': 1,
    'output_channels': 1,
    'hidden_size': 64,
    'train_model':1,
    # Data
    'input_size': [(16, 16, 13),(64, 64, 52)], # 
    'scale_factor': 4,
    'data_split': [0.7, 0, 0.3], 
    'keep_original_scale': True,
    'rescale': True,
}
]

## _3. Choose architecture, dataset and training agent_

- _Dataset\_Nii\_Gz\_3D_ loads 3D files. If you pass a _slice_ it will be split along the according axis.

In [13]:
dataset = Dataset_NiiGz_3D()
device = torch.device(config[0]['device'])
ca1 = BasicNCA3D(config[0]['channel_n'], config[0]['cell_fire_rate'], device, hidden_size=config[0]['hidden_size'], kernel_size=7, input_channels=config[0]['input_channels']).to(device)
ca2 = BasicNCA3D(config[0]['channel_n'], config[0]['cell_fire_rate'], device, hidden_size=config[0]['hidden_size'], kernel_size=3, input_channels=config[0]['input_channels']).to(device)
ca = [ca1, ca2]
agent = Agent_M3D_NCA(ca)
exp = Experiment(config, dataset, ca, agent)
dataset.set_experiment(exp)
exp.set_model_state('train')
data_loader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=exp.get_from_config('batch_size'))

loss_function = DiceFocalLoss() 

NameError: name 'Agent_M3D_NCA' is not defined

## _4. Run training_

In [4]:
agent.train(data_loader, loss_function)

Epoch: 1
Dataset size: 35
1 loss = 1.147585556787603
Epoch: 2
Dataset size: 35
2 loss = 0.8625620245933533
Epoch: 3
Dataset size: 35
3 loss = 0.6668556402711308
Epoch: 4
Dataset size: 35
4 loss = 0.6231558331421443
Epoch: 5
Dataset size: 35
5 loss = 0.5858348704436246
Epoch: 6
Dataset size: 35
6 loss = 0.5380211008446557
Epoch: 7
Dataset size: 35
7 loss = 0.5054158410605263
Epoch: 8
Dataset size: 35
8 loss = 0.5769658601459335
Epoch: 9
Dataset size: 35
9 loss = 0.5127963517393385
Epoch: 10
Dataset size: 35
10 loss = 0.49838484688238666
Evaluate model
_hippocampus_374.nii.gz_0
torch.Size([1, 64, 64, 52, 1]) torch.Size([1, 64, 64, 52, 1])
, 0.7066763639450073
(64, 64, 3)
_hippocampus_326.nii.gz_0
torch.Size([1, 64, 64, 52, 1]) torch.Size([1, 64, 64, 52, 1])
, 0.7221734523773193
(64, 64, 3)
_hippocampus_373.nii.gz_0
torch.Size([1, 64, 64, 52, 1]) torch.Size([1, 64, 64, 52, 1])
, 0.7199017405509949
(64, 64, 3)
_hippocampus_181.nii.gz_0
torch.Size([1, 64, 64, 52, 1]) torch.Size([1, 64, 64, 

Tracking a matplotlib object using "aim.Figure" might not behave as expected.In such cases, consider tracking with "aim.Image".


_hippocampus_320.nii.gz_0
torch.Size([1, 64, 64, 52, 1]) torch.Size([1, 64, 64, 52, 1])
, 0.6353960633277893
(64, 64, 3)
Average Dice Loss 3d: 0, 0.6882157568204201
Standard Deviation 3d: 0, 0.043225504041107525
{'_hippocampus_374.nii.gz_0': 0.7066763639450073, '_hippocampus_326.nii.gz_0': 0.7221734523773193, '_hippocampus_373.nii.gz_0': 0.7199017405509949, '_hippocampus_181.nii.gz_0': 0.7226961255073547, '_hippocampus_330.nii.gz_0': 0.6736212372779846, '_hippocampus_052.nii.gz_0': 0.6914691925048828, '_hippocampus_300.nii.gz_0': 0.7114618420600891, '_hippocampus_046.nii.gz_0': 0.6863368153572083, '_hippocampus_146.nii.gz_0': 0.7384320497512817, '_hippocampus_358.nii.gz_0': 0.6739065647125244, '_hippocampus_006.nii.gz_0': 0.7551707625389099, '_hippocampus_166.nii.gz_0': 0.6848428249359131, '_hippocampus_232.nii.gz_0': 0.6899616718292236, '_hippocampus_386.nii.gz_0': 0.709648072719574, '_hippocampus_057.nii.gz_0': 0.6507531404495239, '_hippocampus_095.nii.gz_0': 0.7136420607566833, '_hi

Tracking a matplotlib object using "aim.Figure" might not behave as expected.In such cases, consider tracking with "aim.Image".


_hippocampus_109.nii.gz_0
torch.Size([1, 64, 64, 52, 1]) torch.Size([1, 64, 64, 52, 1])
, 0.7850872874259949
(64, 64, 3)
_hippocampus_288.nii.gz_0
torch.Size([1, 64, 64, 52, 1]) torch.Size([1, 64, 64, 52, 1])
, 0.7842220067977905
(64, 64, 3)
_hippocampus_320.nii.gz_0
torch.Size([1, 64, 64, 52, 1]) torch.Size([1, 64, 64, 52, 1])
, 0.7438157796859741
(64, 64, 3)
Average Dice Loss 3d: 0, 0.7698288034584563
Standard Deviation 3d: 0, 0.03267186484704073
{'_hippocampus_374.nii.gz_0': 0.8012921810150146, '_hippocampus_326.nii.gz_0': 0.7991384267807007, '_hippocampus_373.nii.gz_0': 0.777942955493927, '_hippocampus_181.nii.gz_0': 0.7713924646377563, '_hippocampus_330.nii.gz_0': 0.7123435735702515, '_hippocampus_052.nii.gz_0': 0.7840486764907837, '_hippocampus_300.nii.gz_0': 0.768379271030426, '_hippocampus_046.nii.gz_0': 0.7867909073829651, '_hippocampus_146.nii.gz_0': 0.8096132278442383, '_hippocampus_358.nii.gz_0': 0.763096034526825, '_hippocampus_006.nii.gz_0': 0.8253231048583984, '_hippocam

Tracking a matplotlib object using "aim.Figure" might not behave as expected.In such cases, consider tracking with "aim.Image".


_hippocampus_109.nii.gz_0
torch.Size([1, 64, 64, 52, 1]) torch.Size([1, 64, 64, 52, 1])
, 0.7905488610267639
(64, 64, 3)
_hippocampus_288.nii.gz_0
torch.Size([1, 64, 64, 52, 1]) torch.Size([1, 64, 64, 52, 1])
, 0.8153061866760254
(64, 64, 3)
_hippocampus_320.nii.gz_0
torch.Size([1, 64, 64, 52, 1]) torch.Size([1, 64, 64, 52, 1])
, 0.7404059767723083
(64, 64, 3)
Average Dice Loss 3d: 0, 0.7841698345491441
Standard Deviation 3d: 0, 0.038154530287000575
{'_hippocampus_374.nii.gz_0': 0.8083895444869995, '_hippocampus_326.nii.gz_0': 0.8191866874694824, '_hippocampus_373.nii.gz_0': 0.8170391321182251, '_hippocampus_181.nii.gz_0': 0.802423894405365, '_hippocampus_330.nii.gz_0': 0.7582886815071106, '_hippocampus_052.nii.gz_0': 0.8082515597343445, '_hippocampus_300.nii.gz_0': 0.7820573449134827, '_hippocampus_046.nii.gz_0': 0.8125848174095154, '_hippocampus_146.nii.gz_0': 0.8255782723426819, '_hippocampus_358.nii.gz_0': 0.7457724809646606, '_hippocampus_006.nii.gz_0': 0.8479173183441162, '_hippo

## _5. Evaluate test data_

In [ ]:
agent.getAverageDiceScore()